## pretrained_models VGG19 

In [1]:
from keras.applications.vgg19 import VGG19
VGGFmodel = VGG19(include_top=True, weights="imagenet",input_shape=(224,224,3))


In [17]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np
from keras import backend as K
import torch
import torchvision
from tqdm import tqdm
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import pandas as pd

In [18]:
import torch

### normalisation 

In [19]:
def normalization(v):
    e=v/v.norm()
    return e

### func qui calcule le vecteur statique

In [20]:
#nb est le nb de image 
# Rbgfeature est le retour de la fc descripteur pour les images d'un acteur et d'une action spécifique concaténé 
def stat(rbgfeature,nb)  :
  
     #cette opération est due à la création d'un vecteur aléatoire lors du l'initialisation
        rbgfeature=rbgfeature[1:nb]
        d=torch.as_tensor(rbgfeature)
        rbgfeature_statM = torch.max(d,0)[0]  
        rbgfeature_statm = torch.min(d,0)[0]
        rbgfeature_Stat =torch.cat((rbgfeature_statm,rbgfeature_statM))
         #ms la concaténation des vecteurs statique max et Min
        return rbgfeature_Stat

### func qui retourne le nb fichier dans un dossier

In [21]:
import pathlib
def countNb(path) :
    initial_count = 0
    for path in pathlib.Path(path).iterdir():
        if path.is_file():
            initial_count += 1

    return(initial_count)

### func qui retourne le vecteur descripteur de du model vgg19

In [22]:
# x c est l image
def descriptor_vggf(x):
# with a Sequential model
    #teste sur le shape de l image
    if (x.shape[1]!=224)or ( x.shape[2] !=224 ):
    #resize l image (224,224)
        x = cv2.resize(x, (224,224), interpolation = cv2.INTER_AREA)
    
    get_2sd_layer_output = K.function([VGGFmodel.layers[0].input],
                                      [VGGFmodel.layers[24].output])
    layer_output = get_2sd_layer_output([x])[0]
    # Layer output est le résultat d'avant-dernière couche du modèle soit la 2e mme couche entièrement connectée 
    b=layer_output
    
    
    return b

### func qui retourne le vecteur descripteur de du model TSN

In [23]:
#x -> image
def descriptor_OP(x):
# with a Sequential model
    
    #normalisation de l image 
    transform_fn = transforms.Compose([
    
    video.VideoToTensor(),
    video.VideoNormalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    #resize l image (224,224)
    a=transforms.Resize((224, 224))
    d=a(x)
    img_list = transform_fn([d.asnumpy()])
    #retourne le résultat d'avant-dernière couche du modèle soit la 2e mme couche entièrement connectée 
    m=net.features(nd.array(img_list[0]).expand_dims(axis=0))
    
    
    
    return m.asnumpy()

### func qui calcule le vecteur statique

In [24]:
# feature est le retoure de la fc desripteur pour les images d'un acteur et d'une action spécifique concaténé 
#nb est le nb de image 
def dyn( feature,nb):
    feature=feature[1:nb]
    # cette opération est due à la création d'un vecteur aléatoire lors du l'initialisation 
    feature_diff = []
    
    for i in range(feature.shape[0] - 4):
                       feature_diff.append(feature[i+4,:] - feature[i,:])
                      
                        d=torch.as_tensor(feature_diff)

    #            
                        dynM = torch.max(d,0)[0]
                        dynm = torch.min(d,0)[0]
                        ms =torch.cat((dynm,dynM))
                #ms la concaténation des vecteurs dynamiques max et Min
                       
    return ms

# VGG19

In [9]:
pathAction=r"C:\Users\amin\extracted patch\Nouveau dossier\S1"
path=r"C:\Users\amin\extracted patch\Nouveau dossier\\"
pathImage=r"C:\Users\amin\extracted patch\action\\"
action = [f[2:] for f in listdir(pathAction) ]
action

['_clapping_HD',
 '_hittingBottle_toLeft_HD',
 '_hittingBottle_toRight_HD',
 '_hittingStick_toLeft_HD',
 '_hittingStick_toRight_HD',
 '_jogging_backView_HD',
 '_jogging_frontView_HD',
 '_jogging_toLeft_sideView_HD',
 '_jogging_toRight_sideView_HD',
 '_kicking_toLeft_HD',
 '_kicking_toRight_HD',
 '_punching_toLeft_sideView_HD',
 '_punching_toRight_sideView_HD',
 '_running_backView_HD',
 '_running_frontView_HD',
 '_running_toLeft_sideView_HD',
 '_running_toRight_sideView_HD',
 '_stabbing_toLeft_HD',
 '_stabbing_toRight_HD',
 '_walking_backView_HD',
 '_walking_frontView_HD',
 '_walking_toLeft_sideView_HD',
 '_walking_toRight_sideView_HD',
 '_wavingHands_HD']

In [10]:
path =r'C:\Users\amin\extracted patch\Nouveau dossier'
pathPatchUb=r'C:\Users\amin\extracted patch\image\ub\S'
pathPatchRh=r'C:\Users\amin\extracted patch\image\rh\S'
pathPatchFb=r'C:\Users\amin\extracted patch\image\fb\S'
pathPatchLh=r'C:\Users\amin\extracted patch\image\lh\S'

In [11]:
classes=[1,2,2,10,10,3,3,3,3,4,4,5,5,6,6,6,6,7,7,8,8,8,8,9]

len(classes)

24

In [12]:
df2 = pd.DataFrame()
dossier = [f for f in listdir(path) ]
m=0 
classes2= []

pathImg = r'C:\Users\amin\extracted patch\action'+'\\'
for i in tqdm(range (5,10)) :
    pathData = r'C:\Users\amin\extracted patch\Nouveau dossier'+'\\'
    pathData=pathData+ dossier[i]+'\\'
    Data = [f for f in listdir(pathData) ]
    mp=dossier[i]
    for j in range (0,len(Data)) :
        a=pathData
        do=Data[j]
        a=a+Data[j]+'\\'

       
        rhand_rbgfeature=np.ndarray(shape=(1,4096))
        lhand_rbgfeature=np.ndarray(shape=(1,4096))
        UB_rbgfeature=np.ndarray(shape=(1,4096))
        FB_rbgfeature=np.ndarray(shape=(1,4096))
      
        nb=int((countNb(a)/2)+1)

        for n in  range(1,nb):
            aRH = pathPatchRh+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            aUB = pathPatchUb+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            aLH = pathPatchLh+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            aFB = pathPatchFb+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'

            #image loaded RH
            imgRH = image.load_img(aRH,color_mode='rgb', target_size=(224, 224))
            RH = image.img_to_array(imgRH)
            RH = np.expand_dims(RH, axis=0)
            RH = preprocess_input(RH)
            #image loaded LH
            imgLH = image.load_img(aLH,color_mode='rgb', target_size=(224, 224))
            LH = image.img_to_array(imgLH)
            LH = np.expand_dims(LH, axis=0)
            LH = preprocess_input(RH)
            #image loaded UB
            imgUB = image.load_img(aUB,color_mode='rgb', target_size=(224, 224))
            UB = image.img_to_array(imgUB)
            UB = np.expand_dims(UB, axis=0)
            UB = preprocess_input(UB)
            #image loaded FB
            imgFB = image.load_img(aFB,color_mode='rgb', target_size=(224, 224))
            FB = image.img_to_array(imgFB)
            FB = np.expand_dims(FB, axis=0)
            FB = preprocess_input(FB)

            
            
            layer_output_RH=descriptor_vggf(RH)
            rhand_rbgfeature=np.append(rhand_rbgfeature, layer_output_RH, axis = 0)
            layer_output_LH=descriptor_vggf(LH)
            lhand_rbgfeature=np.append(lhand_rbgfeature, layer_output_LH, axis = 0)
            layer_output_UB=descriptor_vggf(UB)
            UB_rbgfeature=np.append(UB_rbgfeature, layer_output_UB, axis = 0)
            layer_output_FB=descriptor_vggf(FB)
            FB_rbgfeature=np.append(FB_rbgfeature, layer_output_FB, axis = 0)


        lhand_rbgfeature_Dym=dyn(lhand_rbgfeature,nb)
        lhand_rbgfeature_stat=stat(lhand_rbgfeature,nb)
        
        rhand_rbgfeature_Dym=dyn(rhand_rbgfeature,nb)
        rhand_rbgfeature_stat=stat(rhand_rbgfeature,nb)
        
        UB_rbgfeature_Dym=dyn(UB_rbgfeature,nb)
        UB_rbgfeature_stat=stat(UB_rbgfeature,nb)
        
        FB_rbgfeature_Dym=dyn(FB_rbgfeature,nb)
        FB_rbgfeature_stat=stat(FB_rbgfeature,nb)
        # vapp = vecteur apparence -> le résultat d'une action d'un acteur spécifique 
        Vapp =torch.cat((lhand_rbgfeature_stat,rhand_rbgfeature_stat,UB_rbgfeature_stat,FB_rbgfeature_stat,lhand_rbgfeature_Dym,rhand_rbgfeature_Dym,UB_rbgfeature_Dym,FB_rbgfeature_Dym))
        px = pd.DataFrame(Vapp).astype("float")
        px=px.T
        df2=pd.concat([px, df2])
        classes2.append(do[3:])
            
            
#     df = df.append(Vapp, ignore_index = True)
#     df['classe'] = address

100%|██████████| 5/5 [13:30:21<00:00, 9724.23s/it]   


## pretrained_models TSN 

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model
import pandas as pd

In [10]:
#imporation du pretrained model
net = get_model('vgg16_ucf101', pretrained=True , include_top=False)

In [11]:
from os import listdir
from os.path import isfile, join
from tqdm import tqdm

In [15]:
path =r'C:\Users\amin\extracted patch\Nouveau dossier'
pathPatchUb=r'C:\Users\amin\extracted patch\imageOP\ub\S'
pathPatchRh=r'C:\Users\amin\extracted patch\imageOP\rh\S'
pathPatchFb=r'C:\Users\amin\extracted patch\imageOP\fb\S'
pathPatchLh=r'C:\Users\amin\extracted patch\imageOP\lh\S'

In [29]:
df = pd.DataFrame()
dossier = [f for f in listdir(path) ]
m=0 
classes= []

pathImg = r'C:\Users\amin\extracted patch\optical'+'\\'
for i in tqdm(range (9,10)) :
    pathData = r'C:\Users\amin\extracted patch\Nouveau dossier'+'\\'
    pathData=pathData+ dossier[i]+'\\'
    Data = [f for f in listdir(pathData) ]
    mp=dossier[i]
    for j in tqdm(range (0,len(Data))) :
        a=pathData
        do=Data[j]
        a=a+Data[j]+'\\'
        
       
        rhand_rbgfeature=np.ndarray(shape=(1,4096))
        lhand_rbgfeature=np.ndarray(shape=(1,4096))
        UB_rbgfeature=np.ndarray(shape=(1,4096))
        FB_rbgfeature=np.ndarray(shape=(1,4096))
      
        nb=int((countNb(a)/2))

        for n in  range(1,nb):
            if do+'_'+str(n)=='S3_hittingBottle_toRight_HD_52':
                    continue
                
            aRH = pathPatchRh+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            aUB = pathPatchUb+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            aLH = pathPatchLh+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            aFB = pathPatchFb+mp[1:]+'\\'+do+'_'+str(n)+'.jpg'
            
            #image loaded RH
            imgRH =  image.imread(aRH)
           
           
            #image loaded LH
            imgLH =  image.imread(aLH)
           
            #image loaded UB
            imgUB = image.imread(aUB)
           
            #image loaded FB
            imgFB =  image.imread(aFB)
            

            
            
            layer_output_RH=descriptor_OP(imgRH)
            rhand_rbgfeature=np.append(rhand_rbgfeature, layer_output_RH, axis = 0)
            layer_output_LH=descriptor_OP(imgLH)
            lhand_rbgfeature=np.append(lhand_rbgfeature, layer_output_LH, axis = 0)
            layer_output_UB=descriptor_OP(imgUB)
            UB_rbgfeature=np.append(UB_rbgfeature, layer_output_UB, axis = 0)
            layer_output_FB=descriptor_OP(imgFB)
            FB_rbgfeature=np.append(FB_rbgfeature, layer_output_FB, axis = 0)


        lhand_rbgfeature_Dym=dyn(lhand_rbgfeature,nb)
        lhand_rbgfeature_stat=stat(lhand_rbgfeature,nb)
        
        rhand_rbgfeature_Dym=dyn(rhand_rbgfeature,nb)
        rhand_rbgfeature_stat=stat(rhand_rbgfeature,nb)
        
        UB_rbgfeature_Dym=dyn(UB_rbgfeature,nb)
        UB_rbgfeature_stat=stat(UB_rbgfeature,nb)
        
        FB_rbgfeature_Dym=dyn(FB_rbgfeature,nb)
        FB_rbgfeature_stat=stat(FB_rbgfeature,nb)
        # vapp = vecteur apparence -> le résultat d'une action d'un acteur spécifique 
        Vapp =torch.cat((lhand_rbgfeature_stat,rhand_rbgfeature_stat,UB_rbgfeature_stat,FB_rbgfeature_stat,lhand_rbgfeature_Dym,rhand_rbgfeature_Dym,UB_rbgfeature_Dym,FB_rbgfeature_Dym))
        px = pd.DataFrame(Vapp).astype("float")
        px=px.T
        df=pd.concat([px, df])
        
        classes.append(do[3:])
        
            
            
#     df = df.append(Vapp, ignore_index = True)
#     df['classe'] = address

100%|██████████| 1/1 [6:20:17<00:00, 22817.05s/it]
